## Deliverable 2. Create a Customer Travel Destinations Map.

In [10]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,55.80,66,89,12.75,light rain
1,1,Rikitea,PF,-23.1203,-134.9692,73.33,74,5,21.12,clear sky
2,2,Uyemskiy,RU,64.4743,40.8524,33.85,100,20,2.24,mist
3,3,Broome,US,42.2506,-75.8330,68.38,62,40,1.01,scattered clouds
4,4,Staunton,US,38.1496,-79.0717,68.72,60,1,1.01,clear sky


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
10,10,Victoria,HK,22.2855,114.1577,86.25,77,4,1.01,clear sky
11,11,Wewak,PG,-3.5534,143.6268,81.52,80,86,5.46,overcast clouds
16,16,Avarua,CK,-21.2078,-159.7750,80.65,74,40,10.36,scattered clouds
17,17,Abu Dhabi,AE,24.4667,54.3667,87.91,66,94,3.44,overcast clouds
23,23,Mount Isa,AU,-20.7333,139.5000,89.37,12,96,10.36,overcast clouds
27,27,Buchanan,LR,5.8808,-10.0467,75.16,86,100,5.01,overcast clouds
32,32,Hilo,US,19.7297,-155.0900,82.13,66,40,8.05,scattered clouds
34,34,Isangel,VU,-19.5500,169.2667,82.51,65,40,9.22,scattered clouds
35,35,New Ulm,US,44.3125,-94.4605,75.60,47,1,4.61,clear sky
36,36,Tucumcari,US,35.1717,-103.7250,82.24,20,20,12.66,few clouds


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Victoria,HK,86.25,clear sky,22.2855,114.1577,
11,Wewak,PG,81.52,overcast clouds,-3.5534,143.6268,
16,Avarua,CK,80.65,scattered clouds,-21.2078,-159.7750,
17,Abu Dhabi,AE,87.91,overcast clouds,24.4667,54.3667,
23,Mount Isa,AU,89.37,overcast clouds,-20.7333,139.5000,
27,Buchanan,LR,75.16,overcast clouds,5.8808,-10.0467,
32,Hilo,US,82.13,scattered clouds,19.7297,-155.0900,
34,Isangel,VU,82.51,scattered clouds,-19.5500,169.2667,
35,New Ulm,US,75.60,clear sky,44.3125,-94.4605,
36,Tucumcari,US,82.24,few clouds,35.1717,-103.7250,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found in {hotel_df.loc[index, 'City']}, {hotel_df.loc[index, 'Country']}... skipping.")        

Hotel not found in Adrar, MR... skipping.
Hotel not found in Tucuma, BR... skipping.
Hotel not found in Urubicha, BO... skipping.
Hotel not found in Santa Cruz Del Sur, CU... skipping.
Hotel not found in Thinadhoo, MV... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
#hotel_df[hotel_df['Hotel Name'] == ''].index
clean_hotel_df = hotel_df[hotel_df['Hotel Name'].str.strip().astype(bool)]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Victoria,HK,86.25,clear sky,22.2855,114.1577,Mini Hotel Central
11,Wewak,PG,81.52,overcast clouds,-3.5534,143.6268,In Wewak Boutique Hotel
16,Avarua,CK,80.65,scattered clouds,-21.2078,-159.7750,Paradise Inn
17,Abu Dhabi,AE,87.91,overcast clouds,24.4667,54.3667,Ramee Garden Hotel Apartments
23,Mount Isa,AU,89.37,overcast clouds,-20.7333,139.5000,ibis Styles Mt Isa Verona
...,...,...,...,...,...,...,...
708,Tavares,US,81.00,clear sky,28.8042,-81.7256,Holiday Inn Express & Suites Tavares - Leesbur...
709,Boone,US,82.87,clear sky,39.0334,-92.3335,Cottonwoods RV Park
712,Iberia,US,79.05,few clouds,29.9502,-91.7507,back to back rv park
713,Urucui,BR,84.40,scattered clouds,-7.2294,-44.5561,Hotel Premium Piauí


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))